In [2]:
source("./Functions/MOCL.R") 
source("./Functions/lambda_selection.R")
source("./Functions/MOCL_wcss.R")
source("./Functions/Adaptive_MAM.R") 

source("./Functions/SKM_gap.R")
source("./Functions/evaluation.R")

library(VarSelLCM) # VarselLCM
library(cluster)     # PAM

library(ggplot2)
library(gridExtra)
library(ggpubr)
library(ggthemes)
library(GGally)
library(RColorBrewer)
library(corrplot)
library(dplyr)
library(pdfCluster)
library(fossil)
options(warn=-1)
library(Rtsne)

# Functions

In [3]:
model=MOCL_wcss

In [4]:
# number of K
cor_sta_sub<-function(X,k,sub_rat,seed1=1,seed2=20,num.knotsf=6,max.iter=500){
    kk=k
    
    set.seed(seed1);sub_1=sample(1:nrow(X),nrow(X)*sub_rat)
    set.seed(seed2);sub_2=sample(1:nrow(X),nrow(X)*sub_rat)

    x_1=X[sub_1,]
    x_2=X[sub_2,]

    or_m1=model(Xf=x_1,k=kk,lambda_seq=seq.default(from=0.001,to=3,length=100),delta=0.01) # knot : 6 -> L: 8
    or_m2=model(Xf=x_2,k=kk,lambda_seq=seq.default(from=0.001,to=3,length=100),delta=0.01)
 
    clu_mat=matrix(0,nrow(X),2)
    clu_mat[sub_1,1]=or_m1$cluster
    clu_mat[sub_2,2]=or_m2$cluster
    c_sub=clu_mat[intersect(sub_1,sub_2),] # similarity of intersected subsample
  
    similarity=abs(cor(c_sub[,1],c_sub[,2],method="kendall"))
  
    return(list(corr=similarity,sub1=sub_1,sub2=sub_2,clu_mat=clu_mat))
}

best_k<-function(X,iter_sub=100,K_list=c(3,4,5,6,7),sub_rat=0.8,max.iter=500){
    k_list=K_list
    iter_sub=iter_sub
    data=X
    
    cor_mat=matrix(0,iter_sub,length(k_list))
    colnames(cor_mat)=paste("K",k_list,sep="_")
    for(i in 1:iter_sub){
        for(kk in 1:length(k_list)){
            set_s=round((as.numeric(Sys.time())*10000))
            set_s=as.numeric(set_s%%1000)
            set.seed(set_s)
            ss=round(runif(10,1,10000))
            
            co_f=cor_sta_sub(X=data,k=k_list[kk],sub_rat=sub_rat,seed1=ss[1],seed2=ss[3],
                                 num.knotsf=6,max.iter=500)
            cor_mat[i,kk]=cor_mat[i,kk]+co_f$corr
        
        }
    
    }
    return(cor_mat)
    
}

# Data

In [5]:
library(ggplot2)
dia_data=diamonds

colnames(dia_data)=c('carat','cut','color','clarity','depth','table','price',"length","width","height")

data_x=dia_data[,-7]
price=dia_data[,7]

# Ordinal data labeling
data_x[,2]=(data_x[,2]=="Fair")*1+(data_x[,2]=="Good")*2+(data_x[,2]=="Very Good")*3+
(data_x[,2]=="Premium")*4+(data_x[,2]=="Ideal")*5

data_x[,3]=(data_x[,3]=="D")*1+(data_x[,3]=="E")*2+(data_x[,3]=="F")*3+(data_x[,3]=="G")*4+(data_x[,3]=="H")*5+
(data_x[,3]=="I")*6+(data_x[,3]=="J")*7

data_x[,4]=(data_x[,4]=='I1')*2+(data_x[,4]=='SI1')*3+(data_x[,4]=='SI2')*4+(data_x[,4]=='VS1')*5+
(data_x[,4]=='VS2')*6+(data_x[,4]=='VVS1')*7+(data_x[,4]=='VVS2')*8+(data_x[,4]=='IF')*9-1

data_x=as.matrix(data_x)
data_x=as.data.frame(data_x)

# subsampling
set.seed(133)
sam=sample(1:nrow(data_x),5000)

sub_x=scale(data_x[sam,])
sub_price=t(price[sam,])
y=sub_price

# Optimal K

In [6]:
Sys.time()

[1] "2024-01-31 16:24:30 KST"

In [7]:
K_stab=best_k(sub_x,iter_sub=10,K_list=c(3,4,5,6),sub_rat=0.8)
Sys.time()

[1] "2024-01-31 18:10:30 KST"

In [8]:
round(apply(K_stab,2,mean),4)

K_3    K_4    K_5    K_6 
0.9945 0.9413 0.9872 0.9988

In [11]:
2*round(apply(K_stab,2,sd),4)/sqrt(10)

K_3          K_4          K_5          K_6 
0.0011384200 0.0308638300 0.0110047263 0.0002529822